Importing data manipulation libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Location of Moonjit csv file folder

In [2]:
import os 

main_dir = os.getcwd()

Reading csv file using read_csv

The csv data has different length rows for each entry. Use delimiter to first seperate each entries

In [4]:
data = pd.read_csv(main_dir+'/monojet_Zp2000.0_DM_50.0_chan3.csv', sep='delimiter', engine='python', header=None)

#To print whole content of a file
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print(data.head())

                                                                                                                                                                                                                                                                       0
0  354;monojet_Zp2000.0_DM_50.0;2.44336e-05;832841;-1.22431;b,947383,824498,-0.523103,1.64421;j,258722,243675,0.328962,2.25014;j,520092,108590,-2.2473,-1.85679;j,383024,88405.6,2.14516,-1.95635;j,39507.6,35365.1,0.47046,-1.16445;j,225430,26878.2,-2.81608,-2.25938;
1                                                                                                     354;monojet_Zp2000.0_DM_50.0;2.44336e-05;732115;1.17245;j,1.06946e+06,751597,0.858186,-1.84217;j,457647,110079,2.10287,2.17501;j,49649.1,30875.2,-1.03189,-1.7992;
2                                                         354;monojet_Zp2000.0_DM_50.0;2.44336e-05;1.05628e+06;-3.06744;j,676000,640429,0.33045,0.704554;j,1.2393e+06,602104,1.34324,-0.550856;j,193865,44825

In [20]:
print(data.iloc[0])

0    354;monojet_Zp2000.0_DM_50.0;2.44336e-05;832841;-1.22431;b,947383,824498,-0.523103,1.64421;j,258722,243675,0.328962,2.25014;j,520092,108590,-2.2473,-1.85679;j,383024,88405.6,2.14516,-1.95635;j,39507.6,35365.1,0.47046,-1.16445;j,225430,26878.2,-2.81608,-2.25938;
Name: 0, dtype: object


Seperating data elements having ';' and ',' seperator



In [21]:
seperated_data = pd.DataFrame()

for val in data.values:
  val = pd.Series(val)
  splitted_str = val.str.split(r"[,;]",expand=True)
  seperated_data = seperated_data.append(splitted_str, ignore_index=True)

In [22]:
print(seperated_data.tail())

       0                         1            2       3           4  5   \
6394  354  monojet_Zp2000.0_DM_50.0  2.44336e-05  914355     1.52903  j   
6395  354  monojet_Zp2000.0_DM_50.0  2.44336e-05  782256    -1.20751  j   
6396  354  monojet_Zp2000.0_DM_50.0  2.44336e-05  645619    -2.48399  j   
6397  354  monojet_Zp2000.0_DM_50.0  2.44336e-05  632390   -0.975687  j   
6398  354  monojet_Zp2000.0_DM_50.0  2.44336e-05  820060  -0.0437853  j   

               6            7          8         9  10      11       12  \
6394  1.11994e+06  1.01269e+06   0.453265  -1.77278  b  246762  95229.6   
6395  1.91509e+06       874869    1.41855   1.83927  j  156224   152161   
6396       787964       263158    1.75981  0.456786  j  408087   258903   
6397       731849       722541  -0.134499   2.56393  j  268828   259144   
6398       893689       742040  -0.618057  -3.10466  j  107526   106995   

              13         14 15           16       17        18         19  \
6394     1.60511  -0.

The string split seems to have added an extra empty string at end.

The split operation is done using `expand=True` operation which yeilds an even column length dataset with empty values as `NaN`.

Next operation is to replace empty string with `NaN`


In [23]:
seperated_data.replace('', np.nan, inplace=True)

print(seperated_data.head())

    0                         1            2            3         4  5   \
0  354  monojet_Zp2000.0_DM_50.0  2.44336e-05       832841  -1.22431  b   
1  354  monojet_Zp2000.0_DM_50.0  2.44336e-05       732115   1.17245  j   
2  354  monojet_Zp2000.0_DM_50.0  2.44336e-05  1.05628e+06  -3.06744  j   
3  354  monojet_Zp2000.0_DM_50.0  2.44336e-05       631781   1.64181  j   
4  354  monojet_Zp2000.0_DM_50.0  2.44336e-05       457316  -1.87536  j   

            6       7          8         9  10          11       12        13  \
0       947383  824498  -0.523103   1.64421  j      258722   243675  0.328962   
1  1.06946e+06  751597   0.858186  -1.84217  j      457647   110079   2.10287   
2       676000  640429    0.33045  0.704554  j  1.2393e+06   602104   1.34324   
3       936707  616229   0.973383  -1.56592  j       75998  52304.3  0.905489   
4       640313  589524   0.390749   1.23734  j      332790   126272   1.61998   

          14   15       16       17        18         19   20 

A simple observation of the above data suggests:

* There are total 66 columns, neglecting the empty string column we get 65 as column length 
* First 5 column are identifiers for the collisions data.
* The remaing 65-5=60 columns will be of the jet collision entries.
* Each jet entry is of 5 column length containing `jet name`, `E`, `pt`, `eta` and `phi`
* Thus there will in all 60/5=12 `jet` entries possible.

In [24]:
#Constructing header for the above data format

header_lst = ['event_id', 'proc_id','evnt_wgt','met','metphi']

for i in range(0,12):
  obj = 'obj'+str(i)
  e1 = 'e'+str(i)
  pt = 'pt'+str(i)
  eta = 'eta'+str(i)
  phi = 'phi'+str(i)
  header_lst.append(obj)
  header_lst.append(e1)
  header_lst.append(pt)
  header_lst.append(eta)
  header_lst.append(phi)

header_lst.append('redundant empty string')

print(header_lst)

['event_id', 'proc_id', 'evnt_wgt', 'met', 'metphi', 'obj0', 'e0', 'pt0', 'eta0', 'phi0', 'obj1', 'e1', 'pt1', 'eta1', 'phi1', 'obj2', 'e2', 'pt2', 'eta2', 'phi2', 'obj3', 'e3', 'pt3', 'eta3', 'phi3', 'obj4', 'e4', 'pt4', 'eta4', 'phi4', 'obj5', 'e5', 'pt5', 'eta5', 'phi5', 'obj6', 'e6', 'pt6', 'eta6', 'phi6', 'obj7', 'e7', 'pt7', 'eta7', 'phi7', 'obj8', 'e8', 'pt8', 'eta8', 'phi8', 'obj9', 'e9', 'pt9', 'eta9', 'phi9', 'obj10', 'e10', 'pt10', 'eta10', 'phi10', 'obj11', 'e11', 'pt11', 'eta11', 'phi11', 'redundant empty string']


In [25]:
seperated_data.columns = header_lst

seperated_data.head()

,event_id,proc_id,evnt_wgt,met,metphi,obj0,e0,pt0,eta0,phi0,obj1,e1,pt1,eta1,phi1,obj2,e2,pt2,eta2,phi2,obj3,e3,pt3,eta3,phi3,obj4,e4,pt4,eta4,phi4,obj5,e5,pt5,eta5,phi5,obj6,e6,pt6,eta6,phi6,obj7,e7,pt7,eta7,phi7,obj8,e8,pt8,eta8,phi8,obj9,e9,pt9,eta9,phi9,obj10,e10,pt10,eta10,phi10,obj11,e11,pt11,eta11,phi11,redundant empty string
0,354,monojet_Zp2000.0_DM_50.0,2.44336e-05,832841,-1.22431,b,947383,824498,-0.523103,1.64421,j,258722,243675,0.328962,2.25014,j,520092,108590,-2.2473,-1.85679,j,383024,88405.6,2.14516,-1.95635,j,39507.6,35365.1,0.47046,-1.16445,j,225430,26878.2,-2.81608,-2.25938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,354,monojet_Zp2000.0_DM_50.0,2.44336e-05,732115,1.17245,j,1.06946e+06,751597,0.858186,-1.84217,j,457647,110079,2.10287,2.17501,j,49649.1,30875.2,-1.03189,-1.7992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,354,monojet_Zp2000.0_DM_50.0,2.44336e-05,1.05628e+06,-3.06744,j,676000,640429,0.33045,0.704554,j,1.2393e+06,602104,1.34324,-0.550856,j,193865,44825.7,-2.14231,-0.965933,j,28217.2,24926.6,-0.454264,0.516676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,354,monojet_Zp2000.0_DM_50.0,2.44336e-05,631781,1.64181,j,936707,616229,0.973383,-1.56592,j,75998,52304.3,0.905489,-0.806736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,354,monojet_Zp2000.0_DM_50.0,2.44336e-05,457316,-1.87536,j,640313,589524,0.390749,1.23734,j,332790,126272,1.61998,-1.85524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Next step is to calculate all the `j` jets in the dataset belonging to an event.

Followed by populating table of `e`, `phi`, `eta` and `pt` values from the above extracted dataset.

In [26]:
jet_entries = ['obj','e','pt','eta','phi']

def getlist(dt):
  n=0
  list_of_entries = []
  while n < 12:
    entry_in_row = []
    obj_entry = jet_entries[0]+str(n)

    #If there is no jet entry then neglect the collision event
    if dt[obj_entry] == np.nan:
      return None

    #Only select entries for 'j' jet.
    if dt[obj_entry] == 'j':

      #Each entry is followed by 4 variables E, pt, eta, phi
      for i in range(1,5):
        entry_in_row.append(dt[str(jet_entries[i]+str(n))])
      list_of_entries.append(entry_in_row)
    n+=1
  return list_of_entries


#nn=0
final_dataset = pd.DataFrame()

for ind in range(0, len(seperated_data)):
  
  #Get all entries for a row
  list_of_entries = getlist(seperated_data.iloc[ind])
  
  #There is no jet entry
  if list_of_entries is None:
    continue

  #Insert all jet entries found in a row
  for i in list_of_entries:
    #i.insert(0,int(nn))
    temp = pd.Series(i, index=jet_entries[1:])
    id_entries = seperated_data[header_lst[0:5]].iloc[ind]
    entry = pd.concat([temp, id_entries])
    final_dataset = final_dataset.append(pd.Series(entry), ignore_index=True)

In [27]:
print(final_dataset.head())

         e       eta event_id     evnt_wgt     met    metphi       phi  \
0   258722  0.328962      354  2.44336e-05  832841  -1.22431   2.25014   
1   520092   -2.2473      354  2.44336e-05  832841  -1.22431  -1.85679   
2   383024   2.14516      354  2.44336e-05  832841  -1.22431  -1.95635   
3  39507.6   0.47046      354  2.44336e-05  832841  -1.22431  -1.16445   
4   225430  -2.81608      354  2.44336e-05  832841  -1.22431  -2.25938   

                    proc_id       pt  
0  monojet_Zp2000.0_DM_50.0   243675  
1  monojet_Zp2000.0_DM_50.0   108590  
2  monojet_Zp2000.0_DM_50.0  88405.6  
3  monojet_Zp2000.0_DM_50.0  35365.1  
4  monojet_Zp2000.0_DM_50.0  26878.2  


In [28]:
print('no of unique event entries:', final_dataset['event_id'].nunique())
print('no of unique event weight entries:',final_dataset['evnt_wgt'].nunique())
print('no of unique met entries:', final_dataset['met'].nunique())
print('no of unique metphi entries:', final_dataset['metphi'].nunique())
print('no of unique proc_id entries:', final_dataset['proc_id'].nunique())

len(final_dataset)

1
1
6366
6368
1


22661

Storing data as pickled format for applying autoencoder.

In [29]:
dataset = final_dataset.to_pickle(main_dir+'/processed_datalabel.pkl')

In [30]:
unpickled_data = pd.read_pickle(main_dir+'/processed_datalabel.pkl')

Testing saved file

In [31]:
unpickled_data.head()

,e,eta,event_id,evnt_wgt,met,metphi,phi,proc_id,pt
0,258722,0.328962,354,2.44336e-05,832841,-1.22431,2.25014,monojet_Zp2000.0_DM_50.0,243675
1,520092,-2.2473,354,2.44336e-05,832841,-1.22431,-1.85679,monojet_Zp2000.0_DM_50.0,108590
2,383024,2.14516,354,2.44336e-05,832841,-1.22431,-1.95635,monojet_Zp2000.0_DM_50.0,88405.6
3,39507.6,0.47046,354,2.44336e-05,832841,-1.22431,-1.16445,monojet_Zp2000.0_DM_50.0,35365.1
4,225430,-2.81608,354,2.44336e-05,832841,-1.22431,-2.25938,monojet_Zp2000.0_DM_50.0,26878.2


In [32]:
if 'proc_id' in unpickled_data:
  unpickled_data = unpickled_data.drop(columns=['proc_id'])

unpickled_data = unpickled_data.astype('float64')

#Convert to GeV
unpickled_data['e'] = unpickled_data['e'] / 1000
unpickled_data['pt'] = unpickled_data['pt'] / 1000